# CSII 2024 # Exerercise 09: Seperation Principle, LQG, Stability Margins
&copy; 2024 ETH Zurich, Timm Grigat, Suno Dieckmann, Dejan Milojevic; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli


## Description
TODO: Description of the notebook.

To start, run the following cell to install the necessary modules and import the libraries.

In [2]:
%pip install cs2solutions --force-reinstall


import numpy as np
import control as ct

np.set_printoptions(suppress=True, precision=3)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.1 MB/s eta 0:00:00
  Using cached matplotlib-3.8.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached scipy-1.12.0-cp310-cp310-macosx_12_0_arm64.whl.metadata (112 kB)
  Using cached contourpy-1.2.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Using cached kiwisolver-1.4.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.5 MB/s eta 0:00:00-

In [112]:
def hinf_norm(system: ct.TransferFunction) -> int:
    """Calculate the h_inf_norm of a given systems transfer function."""
    # Calculate frequency response over a wide range of frequencies
    omega = np.linspace(-4, 4, 1000)
    H = system(omega * 1j)
    # Calculate all the singular values after checking for MIMO
    if system.ninputs > 1 or system.noutputs > 1:
        singular_values = [np.linalg.svd(H[..., i], compute_uv=False)
                        for i in range(len(omega))]
    else:
        singular_values = [np.absolute(H[... , i]) for i in range(len(omega))]
    # Return the highest singular value
    return np.vstack(singular_values).max()

# Exercise 1: Small gain Theorem

Consider the following System with three plants(P0, P1, P2)
Write a function to check if the system is stable using the small gain theorem.
You have the following functions at hand:
- __np.hinf_norm(transfer_function)__ which calculates the h infinity norm from a given lti system e.g. _np.hinf_norm([1],[1,1]) = __
- __get_transfer_function(system)__ which gives you the transfer function of the subsystems e.g. get_transfer_function(0) for P0.

Write a function that returns a boolean depending on the stability of the system.


In [ ]:
def small_gain_check(P0: int, P1: int) -> bool:
    """Checks if the system is stable depending on the given subsystems."""
    
    # TODO: Write a function to check for the stability of the system.
    
    return True



# Exercise 2: Stability
Think of a cool exercise.

In [ ]:
def stability_check() -> bool:

#TODO: Write a function that checks the internal and external stability of the system.

    return True

# Exercise 3: Performance Robustness
Think of a cool exercise.